# Reconstructing RSA Private Keys from Random Key Bits

A pure sage implementation of papar : [Reconstructing RSA Private Keys from Random Key Bits](https://eprint.iacr.org/2008/510.pdf).

The attack from the papper is able to make use of five components of the RSA private key: $p, q, d, d_p$, and $d_q$. We can use known bits in $d, d_p$, and $d_q$ to make progress where bits in $p$ and $q$ are not known. To relate $d$ to the rest of the private key, we make use of techniques due to Boneh, Durfee, and Frankel; to relate $d_p$ and $d_q$ to the rest of the private key, we make new observations about the structure of RSA keys that may be of independent interest.

If the algorithm has access to fewer components of the RSA private key, the algorithm will still perform well given a sufficiently large fraction of the bits. For example, it can efficiently recover a key given :

- $\delta=.27$ fraction of the bits of $p, q, d, d_p$, and $d_q$.
- $\delta=.42$ fraction of the bits of $p, q$, and $d$.
- $\delta=.57$ fraction of the bits of $p$ and $q$.


**Remarks**

The analysis is manily (modified) from the original paper.

## RSA private key leak model

I write a demo function that simulates a real-world scenario of RSA private key exposure.

In [1]:
from Crypto.Util.number import getPrime, inverse
from random import sample
from itertools import product

def gen_rsa_priv_key(nbits, ebits=None):
    n = int(1)
    while n.bit_length() != nbits:
        p = getPrime(nbits//2)
        q = getPrime(nbits//2)
        n = p * q
    if ebits is None:
        e = 65537
    else:
        e = getPrime(ebits)
    d = inverse(e, (p-1)*(q-1))
    dp = d % (p-1)
    dq = d % (q-1)
    qinv = inverse(q, p)
    return (n, e, d, p, q, dp, dq, qinv)

def rsa_random_leak_model(priv_key, p_per, q_per, d_per, dp_per, dq_per):
    n, e, d, p, q, dp, dq, qinv = priv_key
    nbits = int(n).bit_length()
    L = list(range(nbits))
    L2 = list(range(nbits//2))
    leak_d_bits = sample(L, int(nbits * d_per))
    leak_p_bits = sample(L2, int(nbits//2 * p_per))
    leak_q_bits = sample(L2, int(nbits//2 * q_per))
    leak_dp_bits = sample(L2, int(nbits//2 * dp_per))
    leak_dq_bits = sample(L2, int(nbits//2 * dq_per))
    leak_d = {}
    for pos in leak_d_bits:
        leak_d[pos] = (d >> pos) & 1
    leak_p = {}
    for pos in leak_p_bits:
        leak_p[pos] = (p >> pos) & 1
    leak_q = {}
    for pos in leak_q_bits:
        leak_q[pos] = (q >> pos) & 1
    leak_dp = {}
    for pos in leak_dp_bits:
        leak_dp[pos] = (dp >> pos) & 1
    leak_dq = {}
    for pos in leak_dq_bits:
        leak_dq[pos] = (dq >> pos) & 1
    return (leak_d, leak_p, leak_q, leak_dp, leak_dq)

priv_key = gen_rsa_priv_key(1024)
(leak_d, leak_p, leak_q, leak_dp, leak_dq) = rsa_random_leak_model(priv_key, 0.01, 0.01, 0.01, 0.01,  0.01)
print(f"[+] The leak dict of d is : {leak_d = }")

[+] The leak dict of d is : leak_d = {264: 1, 445: 1, 352: 1, 737: 0, 650: 0, 678: 0, 981: 1, 849: 1, 814: 1, 935: 0}


## Computing k

We have four equations about $n,e,p,q,d,d_p,d_q$ :

$$
\begin{aligned}
n & = p q & (1)\\
e d & =k(N-p-q+1)+1 & (2)\\
e d_p & =k_p(p-1)+1 & (3)\\
e d_q & =k_q(q-1)+1 & (4)
\end{aligned}
$$

The following argument, due to Boneh, Durfee, and Frankel, shows that $k$ must be in the range $0<k<e$. We know $d<\varphi(N)$. Assume $e \leq k$; then $e d<k \varphi(N)+1$, which contradicts (2). The case $k=0$ is also impossible, as can be seen by reducing (2) modulo $e$. This shows that we can enumerate all possible values of $k$, having assumed that $e$ is small.
For each such choice $k^{\prime}$, define
$$
\tilde{d}\left(k^{\prime}\right) \stackrel{\text { def }}{=}\left\lfloor\frac{k^{\prime}(N+1)+1}{e}\right\rfloor .
$$

As Boneh, Durfee, and Frankel observe, when $k^{\prime}$ equals $k$, this gives an **excellent approximation** for $d$ :
$$
0 \leq \tilde{d}(k)-d \leq k(p+q) / e<p+q .
$$

In particular, when $p$ and $q$ are balanced, we have $p+q<3 \sqrt{N}$, which means that $\tilde{d}(k)$ agrees with $d$ on their $\lfloor n / 2\rfloor-2$ most significant bits (less when p and q are not so balanced). (Our analysis applies also in the less common case when $p$ and $q$ are unbalanced, but we omit the details.) This means that small-public-exponent RSA leaks half the bits of the private exponent in one of the candidate values $\tilde{d}(1), \ldots, \tilde{d}(e-1)$.


**A demo function that can recover validate $k$ from the known msb bits from $d$. After $k$ is recovered, we can recover more msb bits of $d$ (approximately half msbs of $d$).**


In [2]:
def generate_k_d_pair(n,e):
    # e * d = k(p-1)(q-1) + 1 = k(n - p - q + 1) + 1
    # Given n and small e, compute all k candidates and its corresponding d
    assert e <= 2**20, "e is too large, consider using C implementation"
    for k in range(1, e):
        d_ = (k * n + 1) // e
        yield k, d_
        
def compute_k_d_pair(n, e, leak_d, start_pos=None):
    # p, q must be balanced : p < q < 2p
    # then d_ = (k * n + 1) // e agrees with d on their floor[nbits/2] - 2 msbs, omit extra 2 bits
    if start_pos == None:
        start_pos = int(n).bit_length()//2 + 2
    leak_d_highbits = {pos for pos in leak_d if pos >= start_pos}
    for k, d in generate_k_d_pair(n, e):
        if all((d >> pos) & 1 == leak_d[pos] for pos in leak_d_highbits):
            yield k, d
            
def test_recovering_k(leak_percent = 0.3):
    n, e, d, p, q, dp, dq, qinv = gen_rsa_priv_key(1024)
    k = (e * d - 1) // ((p-1)*(q-1))
    leak_d, leak_p, leak_q, leak_dp, leak_dq = rsa_random_leak_model((n, e, d, p, q, dp, dq, qinv), leak_percent, leak_percent, leak_percent, leak_percent, leak_percent)
    pos = int(n).bit_length()//2 + 32
    L = list(compute_k_d_pair(n, e, leak_d))
    print(f"Recovered {len(L) = }")
    if len(L) == 0:
        print(f"Failed to recover k, try with pos = {pos}")
        L = list(compute_k_d_pair(n, e, leak_d, pos))
        print(f"Recovered {len(L) = }, {L = }")

test_recovering_k(0.1)

Recovered len(L) = 1


## Computing $k_p$ and $k_q$. 

Once we have determined $k$, we can compute $k_p$ and $k_q$. First, observe that by an analysis like that above, we can show that $0<k_p, k_q<e$. This, of course, means that $k_p=\left(k_p \bmod e\right)$ and $k_q=\left(k_q \bmod e\right)$; when we solve for $k_p$ and $k_q$ modulo $e$, this will reveal the actual values used in (3) and (4). Now, reducing equations (1) -(4) modulo $e$, we obtain the following congruences:
$$
\begin{aligned}
N & \equiv p q  & (5)\\
0 & \equiv k(N-p-q+1)+1 & (6) \\
0 & \equiv k_p(p-1)+1 & (7)\\
0 & \equiv k_q(q-1)+1 & (8)
\end{aligned}
$$

These are four congruences in four unknowns: $p, q, k_p$, and $k_q$; we solve them as follows. From (7) and (8) we write $(p-1) \equiv-1 / k_p$ and $(q-1) \equiv-1 / k_q$; we substitute these into the equation obtained from using (5) to reexpress $\varphi(N)$ in (6): $0 \equiv k(N-p-q+1)+1 \equiv k(p-1)(q-1)+1 \equiv$ $k\left(-1 / k_p\right)\left(-1 / k_q\right)+1 \equiv k /\left(k_p k_q\right)+1$, or
$$
k+k_p k_q \equiv 0 \quad (9)
$$

Next, we return to (6), substituting in (7), (8), and (9):
$$
\begin{aligned}
0 & \equiv k(N-p-q+1)+1 \\
& \equiv k(N-1)-k(p-1+q-1)+1 \\
& \equiv k(N-1)-\left(-k_p k_q\right)\left(-1 / k_p-1 / k_q\right)+1 \\
& \equiv k(N-1)-\left(k_q+k_p\right)+1 ;
\end{aligned}
$$
we solve for $k_p$ by substituting $k_q=-k / k_p$, obtaining
$$
0 \equiv k(N-1)-\left(k_p-k / k_p\right)+1,
$$
or, multiplying both sides by $k_p$ and rearranging,
$$
k_p^2-[k(N-1)+1] k_p-k \equiv 0 \quad (10)
$$

This congruence is easy to solve modulo $e$ and, in the common case where $e$ is prime, has two solutions, just as it would over $\mathbb{C}$. One of the two solutions is the correct value of $k_p$; and it is easy to see, by symmetry, that the other must be the correct value of $k_q$. We need therefore try just two possible assignments to $k_p$ and $k_q$ in reconstructing the RSA key. When $e$ has $m$ distinct prime factors, there may be up to $2^m$ roots.

In [3]:
def compute_kp_kq(n, e, k):
    # Given n, e and k, compute kp and kq such that
    # e * d = 1 + k(p-1)(q-1) = 1 + k * (n - p - q + 1)
    # e * dp = kp * (p-1) + 1
    # e * dq = kq * (q-1) + 1
    # n = p * q
    # solve x for eq: kp^2 - [k*(n-1) + 1] * kp - k = 0 \mod e
    # e should be small and prime (or product of several primes)
    x = var('x')
    sols = solve_mod([x**2 - (k*(n-1) + 1)*x - k == 0], e)
    return [int(sol[0]) for sol in sols]

## Recover more information from the known leaks

We assume that we know the values of $k_p$ and $k_q$. When equation (10) has two distinct solutions, we must run the algorithm twice, once for each of the possible assignments to $k_p$ and $k_q$.

Let $p[i]$ denote the $i$ th bit of $p$, where the least significant bit is bit 0 , and similarly index the bits of $q, d, d_p$ and $d_q$. Let $\tau(x)$ denote the exponent of the largest power of 2 that divides $x$.

As $p$ and $q$ are large primes, we know they are odd, so we can correct $p[0]=q[0]=1$. It follows that $2 \mid p-1$, so $2^{1+\tau\left(k_p\right)} \mid k_p(p-1)$. Thus, reducing (3) modulo $2^{1+\tau\left(k_p\right)}$, we have
$$
e d_p \equiv 1 \quad\left(\bmod 2^{1+\tau\left(k_p\right)}\right) .
$$

Since we know $e$, this allows us immediately to correct the $1+\tau\left(k_p\right)$ least significant bits of $d_p$. Similar arguments using (4) and (2) allow us to correct the $1+\tau\left(k_q\right)$ and $2+\tau(k)$ bits of $d_q$ and $d$, respectively.

We can also find the correctc $k_p, k_q$ from the known leaks of $d_p, d_q$. This is a demo that can collect information to the fullest extent possible.

In [4]:
def tau(x):
    r = 0
    while x % 2 == 0:
        x = x // 2
        r += 1
    return int(r)

def recover_info_from_rsa_priv_leaks(n, e, leaks):
    assert e % 2 == 1 and e > 2, "e must be odd and greater than 2"
    d_leak, p_leak, q_leak, dp_leak, dq_leak = leaks
    # copy leaks to avoid modifying the original leaks
    d_bits, p_bits, q_bits, dp_bits, dq_bits = d_leak.copy(), p_leak.copy(), q_leak.copy(), dp_leak.copy(), dq_leak.copy()
    nbits = int(n).bit_length()
    # Recover k
    if d_leak == None:
        return None
    
    pos = nbits // 2 + 2
    k_d = list(compute_k_d_pair(n, e, d_leak, pos))
    while len(k_d) == 0:
        pos += 5
        k_d = list(compute_k_d_pair(n, e, d_leak, pos))
        
    if len(k_d) > 1:
        print(f"[+] Warning: Multiple k candidates found: {len(k_d) = }")
    k, d_ = k_d[0]
    print(f"[+] Recovered k = {k}")
    kps = compute_kp_kq(n, e, k)
    # init all information we can derive from the known leaks
    for i in range(pos, nbits):
        if i in d_leak:
            assert (d_ >> i) & 1 == d_leak[i], "Inconsistent d leaks"
        else:
            d_bits[i] = (d_ >> i) & 1
    
    # lsb leaks of p, q, dp, dq, d
    p_bits[0] = 1
    q_bits[0] = 1
    kp_kq_s = product(kps, repeat=2)
    
    # lsb of d 
    tau_k = tau(k)
    d_lsb = inverse(e, 2**(2 + tau_k))
    for i in range(2 + tau_k):
        if i in d_leak:
            assert (d_lsb >> i) & 1 == d_leak[i], "Inconsistent d leaks"
        else:
            d_bits[i] = (d_lsb >> i) & 1
    
    for kp, kq in kp_kq_s:
        dp_bits = dp_leak.copy()
        dq_bits = dq_leak.copy()
        if kp == kq:
            # not considering kp == kq case
            continue
        tau_kp = tau(kp)
        tau_kq = tau(kq)
        dp_lsb = inverse(e, 2**(1 + tau_kp))
        dq_lsb = inverse(e, 2**(1 + tau_kq))
        good_guess = True
        for i in range(1 + tau_kp):
            if i in dp_leak:
                if (dp_lsb >> i) & 1 != dp_leak[i]:
                    # "Inconsistent dp leaks"
                    good_guess = False
                    break
            else:
                dp_bits[i] = (dp_lsb >> i) & 1
        if not good_guess:
            continue
        for i in range(1 + tau_kq):
            if i in dq_leak:
                if (dq_lsb >> i) & 1 != dq_leak[i]:
                    # "Inconsistent dq leaks"
                    good_guess = False
                    break
            else:
                dq_bits[i] = (dq_lsb >> i) & 1
        if good_guess:
            yield (n, e, k, kp, kq), (d_bits, p_bits, q_bits, dp_bits, dq_bits), (tau_k, tau_kp, tau_kq)

## Reconstructing RSA 

What is more, we can easily see that, having fixed bits $<i$ of $p$, a change in $p[i]$ affects $d_p$ not in bit $i$ but in bit $i+\tau\left(k_p\right)$; and, similarly, a change in $q[i]$ affects $d_q\left[i+\tau\left(k_q\right)\right]$, and a change in $p[i]$ or $q[i]$ affects $d[i+\tau(k)]$. When any of $k, k_p$, or $k_q$ is odd, this is just the trivial statement that changing bit $i$ of the right-hand side of an equation changes bit $i$ of the left-hand side. Powers of 2 in $k_p$ shift left the bit affected by $p[i]$, and similarly for the other variables.

Having recovered the least-significant bits of each of our five variables, we now attempt to recover the remaining bits. For each bit index $i$, we consider a slice of bits:
$$
p[i] \quad q[i] \quad d[i+\tau(k)] \quad d_p\left[i+\tau\left(k_p\right)\right] \quad d_q\left[i+\tau\left(k_q\right)\right] .
$$

For each possible solution up to bit slice $i-1$, generate all possible solutions up to bit slice $i$ that agree with that solution at all but the $i$ th position. If we do this for all possible solutions up to bit slice $i-1$, we will have enumerated all possible solutions up to bit slice $i$. Above, we already described how to obtain the only possible solution up to $i=0$; this is the solution we use to start the algorithm. The factorization of $N$ will be revealed in one or more of the possible solutions once we have reached $i=\lfloor n / 2\rfloor^3$

All that remains is how to lift a possible solution $\left(p^{\prime}, q^{\prime}, d^{\prime}, d_p^{\prime}, d_q^{\prime}\right)$ for slice $i-1$ to possible solutions for slice $i$. Naïvely there are $2^5=32$ such possibilities, but in fact there are at most 2 and, for large enough $\delta$, almost always fewer.

First, observe that we have four constraints on the five variables: equations (1), (2), (3), and (4). By plugging in the values up to slice $i-1$, we obtain from each of these a constraint on slice $i$, namely values $c_1, \ldots, c_4$ such that the following congruences hold modulo 2 :
$$
\begin{aligned}
& p[i]+q[i] \equiv c_1 \quad(\bmod 2) \\
& d[i+\tau(k)]+p[i]+q[i] \equiv c_2 \quad(\bmod 2) \\
& d_p\left[i+\tau\left(k_p\right)\right]+p[i] \equiv c_3 \quad(\bmod 2) \\
& d_q\left[i+\tau\left(k_q\right)\right]+q[i] \equiv c_4 \quad(\bmod 2) \\
\end{aligned} \quad (11)
$$


**Lifting solutions $\bmod 2^i$.** The process of generating bit $i$ of a partial solution given bits 0 through $i-1$ can be seen as lifting a solution to the constraint equations $\bmod 2^i$ to a solution $\bmod$ $2^{i+1}$. Hensel's lemma characterizes the conditions when this is possible.

**Lemma 4.2 (Multivariate Hensel's Lemma).** A root $\mathbf{r}=\left(r_1, r_2, \ldots, r_n\right)$ of the polynomial $f\left(x_1, x_2, \ldots, x_n\right) \bmod \pi^i$ can be lifted to a root $\mathbf{r}+\mathbf{b} \bmod \pi^{i+1}$ if $\mathbf{b}=\left(b_1 \pi^i, b_2 \pi^i, \ldots, b_n \pi^i\right), 0 \leq$ $b_j \leq \pi-1$ is a solution to the equation
$$
f(\mathbf{r}+\mathbf{b})=f(\mathbf{r})+\sum_j b_j \pi^i f_{x_j}(\mathbf{r}) \equiv 0 \quad\left(\bmod \pi^{i+1}\right) .
$$
(Here, $f_{x_j}$ is the partial derivative of $f$ with respect to $x_j$.)
We can rewrite the lemma using the notation of Section 3. Write $\mathbf{r}$ in base $\pi=2$ and assume the $i$ first bits are known. Then the lemma tells us that the next bit of $\mathbf{r}, \mathbf{r}[i]=\left(r_1[i], r_2[i], \ldots\right)$, must satisfy
$$
f(\mathbf{r})[i]+\sum_j f_{x_j}(\mathbf{r}) r_j[i] \equiv 0 \quad(\bmod 2) . \quad (12)
$$

In our case, the constraint polynomials generated in Section 2, equations (1)-(4) form four simultaneous equations in five variables. Given a partial solution $\left(p^{\prime}, q^{\prime}, d^{\prime}, d_p^{\prime}, d_q^{\prime}\right)$ up to slice $i$ of the bits, we apply the condition in equation (12) above to each polynomial and reduce modulo 2 to obtain the following conditions on bit $i$ :
$$
\begin{array}{rlrl}
p[i]+q[i] & \equiv\left(n-p^{\prime} q^{\prime}\right)[i] & & (\bmod 2) \\
d[i+\tau(k)]+p[i]+q[i] & \equiv\left(k(N+1)+1-k\left(p^{\prime}+q^{\prime}\right)-e d^{\prime}\right)[i+\tau(k)] & & (\bmod 2) \\
d_p\left[i+\tau\left(k_p\right)\right]+p[i] & \equiv\left(k_p\left(p^{\prime}-1\right)+1-e d_p^{\prime}\right)\left[i+\tau\left(k_p\right)\right] & & (\bmod 2) \\
d_q\left[i+\tau\left(k_q\right)\right]+q[i] & \equiv\left(k_q\left(q^{\prime}-1\right)+1-e d_q^{\prime}\right)\left[i+\tau\left(k_q\right)\right] & & (\bmod 2) 
\end{array}
$$

These are precisely (11).



In [5]:
def bit_i(x, i):
    return (x >> i) & 1
            
def reconstructing_rsa_priv_key_v0(n, e, leaks):
    iter_table = list(product([0, 1], repeat=5))
    
    def sub_process(pk, known_bits, taus, p, q, d, dp, dq, pos):
        (n, e, k, kp, kq), (d_bits, p_bits, q_bits, dp_bits, dq_bits), (tau_k, tau_kp, tau_kq) = pk, known_bits, taus
        if pos == nbits//2 and (n % p == 0 or n % q == 0):
            print("[+] RSA private key recovered !")
            print(f"[+] {p = }")
            print(f"[+] {q = }")
            return (p, q, d, dp, dq)
        i = pos
        for pi, qi, dpi, dqi, di in iter_table:
            # the known bits            
            if i in p_bits and pi != p_bits[i]:
                continue
            if i in q_bits and qi != q_bits[i]:
                continue
            if (i + tau_kp) in dp_bits and dpi != dp_bits[i + tau_kp]:
                continue
            if (i + tau_kq) in dq_bits and dqi != dq_bits[i + tau_kq]:
                continue
            if (i + tau_k) in d_bits and di != d_bits[i + tau_k]:
                continue
            # the 4 equations
            if (pi + qi) % 2 != bit_i(n- p*q, i):
                continue
            if (di + pi + qi) % 2 != bit_i(k * (n + 1) + 1 - k *(p + q) - e * d, i + tau_k):
                continue
            if (dpi + pi) % 2 != bit_i(kp * (p - 1) + 1 - e * dp, i + tau_kp):
                continue
            if (dqi + qi) % 2 != bit_i(kq * (q - 1) + 1 - e * dq, i + tau_kq):
                continue
            # yield from sub_process(n,e,p,q,d,dp,dq,i+1)
            new_p = p | (pi << i)
            new_q = q | (qi << i)
            new_d = d | (di << (i + tau_k))
            new_dp = dp | (dpi << (i + tau_kp))
            new_dq = dq | (dqi << (i + tau_kq))
            result = sub_process(pk, known_bits, taus, new_p, new_q, new_d, new_dp, new_dq, i+1)
            if result is not None:
                return result  
        return None
                
    for pk, known_bits, taus in recover_info_from_rsa_priv_leaks(n, e, leaks):
        (n, e, k, kp, kq), (d_bits, p_bits, q_bits, dp_bits, dq_bits), (tau_k, tau_kp, tau_kq) = pk, known_bits, taus
        nbits = int(n).bit_length()
        print(f"[+] Try with {kp = }, {kq = }")
        p = 1
        q = 1
        d = ZZ([d_bits[i] for i in range(1 + tau_k)], base=2)
        dp = ZZ([dp_bits[i] for i in range(1 + tau_kp)], base=2)
        dq = ZZ([dq_bits[i] for i in range(1 + tau_kq)], base=2)
        res = sub_process(pk, known_bits, taus, p, q, d, dp, dq, 1)
        if res is not None:
            return res
        

The above function is not optimized. And if $n$ is big or the leak bits are insufficient, there is a possibility of overflowing the maximum recursion depth. There is a bug of `setrecursionlimit` in sage (at least in my env and irreparable). Therefore, I optimized the codes and write a iteration version instead of recursion.

In [6]:
def _reconstructing_rsa_priv_key(pk, known_bits, taus):
    iter_table = list(product([0, 1], repeat=5))
    def core_process(p, q, d, dp, dq, pos):
        if pos == max_pos and (n % p == 0 or n % q == 0):
            print("[+] RSA private key recovered !")
            print(f"[+] {p = }")
            print(f"[+] {q = }")
            return (p, q, d, dp, dq)
        i = pos
        for pi, qi, dpi, dqi, di in iter_table:
            # the known bits            
            if i in p_bits and pi != p_bits[i]:
                continue
            if i in q_bits and qi != q_bits[i]:
                continue
            if (i + tau_kp) in dp_bits and dpi != dp_bits[i + tau_kp]:
                continue
            if (i + tau_kq) in dq_bits and dqi != dq_bits[i + tau_kq]:
                continue
            if (i + tau_k) in d_bits and di != d_bits[i + tau_k]:
                continue
            # the 4 equations
            if (pi + qi) % 2 != bit_i(n- p*q, i):
                continue
            if (di + pi + qi) % 2 != bit_i(k * (n + 1) + 1 - k *(p + q) - e * d, i + tau_k):
                continue
            if (dpi + pi) % 2 != bit_i(kp * (p - 1) + 1 - e * dp, i + tau_kp):
                continue
            if (dqi + qi) % 2 != bit_i(kq * (q - 1) + 1 - e * dq, i + tau_kq):
                continue
            # yield from sub_process(n,e,p,q,d,dp,dq,i+1)
            new_p = p | (pi << i)
            new_q = q | (qi << i)
            new_d = d | (di << (i + tau_k))
            new_dp = dp | (dpi << (i + tau_kp))
            new_dq = dq | (dqi << (i + tau_kq))
            result = core_process(new_p, new_q, new_d, new_dp, new_dq, i+1)
            if result is not None:
                return result
        return None
    
    (n, e, k, kp, kq), (d_bits, p_bits, q_bits, dp_bits, dq_bits), (tau_k, tau_kp, tau_kq) = pk, known_bits, taus
    nbits = int(n).bit_length()
    max_pos = nbits//2
    print(f"[+] Try with {kp = }, {kq = }")
    p = 1
    q = 1
    d = int(ZZ([d_bits[i] for i in range(1 + tau_k)], base=2))
    dp = int(ZZ([dp_bits[i] for i in range(1 + tau_kp)], base=2))
    dq = int(ZZ([dq_bits[i] for i in range(1 + tau_kq)], base=2))
    return core_process(p, q, d, dp, dq, 1)

def _reconstructing_rsa_priv_key_iter(pk, known_bits, taus):
    from collections import deque 
    iter_table = list(product([0, 1], repeat=5))
    (n, e, k, kp, kq), (d_bits, p_bits, q_bits, dp_bits, dq_bits), (tau_k, tau_kp, tau_kq) = pk, known_bits, taus
    nbits = int(n).bit_length()
    max_pos = nbits//2
    print(f"[+] Try with {kp = }, {kq = }")
    p = 1
    q = 1
    d = int(ZZ([d_bits[i] for i in range(1 + tau_k)], base=2))
    dp = int(ZZ([dp_bits[i] for i in range(1 + tau_kp)], base=2))
    dq = int(ZZ([dq_bits[i] for i in range(1 + tau_kq)], base=2))
    stack = deque()
    stack.append((p, q, d, dp, dq, 1))
    # not empty
    while stack:
        p, q, d, dp, dq, pos = stack.pop()
        if pos == max_pos:
            if (n % p == 0 or n % q == 0):
                print("[+] RSA private key recovered !")
                print(f"[+] {p = }")
                print(f"[+] {q = }")
                return (p, q, d, dp, dq)
            else:
                continue
        i = pos
        for pi, qi, dpi, dqi, di in iter_table:
            # the known bits            
            if i in p_bits and pi != p_bits[i]:
                continue
            if i in q_bits and qi != q_bits[i]:
                continue
            if (i + tau_kp) in dp_bits and dpi != dp_bits[i + tau_kp]:
                continue
            if (i + tau_kq) in dq_bits and dqi != dq_bits[i + tau_kq]:
                continue
            if (i + tau_k) in d_bits and di != d_bits[i + tau_k]:
                continue
            # the 4 equations
            if (pi + qi) % 2 != bit_i(n- p*q, i):
                continue
            if (di + pi + qi) % 2 != bit_i(k * (n + 1) + 1 - k *(p + q) - e * d, i + tau_k):
                continue
            if (dpi + pi) % 2 != bit_i(kp * (p - 1) + 1 - e * dp, i + tau_kp):
                continue
            if (dqi + qi) % 2 != bit_i(kq * (q - 1) + 1 - e * dq, i + tau_kq):
                continue
            # yield from sub_process(n,e,p,q,d,dp,dq,i+1)
            new_p = p | (pi << i)
            new_q = q | (qi << i)
            new_d = d | (di << (i + tau_k))
            new_dp = dp | (dpi << (i + tau_kp))
            new_dq = dq | (dqi << (i + tau_kq))
            stack.append((new_p, new_q, new_d, new_dp, new_dq, i+1))
    return None

def _reconstructing_rsa_priv_key_iter_pq(pk, known_bits):
    # with only leaks of p and q
    from collections import deque 
    iter_table = list(product([0, 1], repeat=2))
    (n, e), (p_bits, q_bits) = pk, known_bits
    nbits = int(n).bit_length()
    max_pos = nbits//2
    p = 1
    q = 1
    stack = deque()
    stack.append((p, q, 1))
    # not empty
    while stack:
        p, q, pos = stack.pop()
        if pos == max_pos:
            if (n % p == 0 or n % q == 0):
                print("[+] RSA private key recovered !")
                print(f"[+] {p = }")
                print(f"[+] {q = }")
                return (p, q)
            else:
                continue
        i = pos
        for pi, qi in iter_table:
            # the known bits            
            if i in p_bits and pi != p_bits[i]:
                continue
            if i in q_bits and qi != q_bits[i]:
                continue
            # the equations
            if (pi + qi) % 2 != bit_i(n- p*q, i):
                continue
            new_p = p | (pi << i)
            new_q = q | (qi << i)
            stack.append((new_p, new_q, i+1))
    return None
        
def reconstructing_rsa_priv_key(n, e, leaks):            
    for pk, known_bits, taus in recover_info_from_rsa_priv_leaks(n, e, leaks):
        res = _reconstructing_rsa_priv_key(pk, known_bits, taus)
        if res is not None:
            return res
    return None

def reconstructing_rsa_priv_key_iter(n, e, leaks):        
    for pk, known_bits, taus in recover_info_from_rsa_priv_leaks(n, e, leaks):
        res = _reconstructing_rsa_priv_key_iter(pk, known_bits, taus)
        if res is not None:
            return res
    return None

def reconstructing_rsa_priv_key_iter_pq(n, e, leaks):        
    return _reconstructing_rsa_priv_key_iter_pq((n,e), leaks)

In [7]:
def test_reconstructing_rsa_priv_key():
    n, e, d, p, q, dp, dq, qinv = gen_rsa_priv_key(2048)
    print("[+] RSA Key information")
    print(f"[+] {p = }")
    print(f"[+] {q = }")
    k = (e * d - 1) // ((p-1)*(q-1))
    kp = (e * dp - 1) // (p-1)
    kq = (e * dq - 1) // (q-1)
    print(f"[+] {k = }")
    print(f"[+] {kp = }")
    print(f"[+] {kq = }")
    print()
    leaks = rsa_random_leak_model((n, e, d, p, q, dp, dq, qinv), 0.3, 0.3, 0.3, 0.3, 0.3)
    print("[+] Try iter version p q d dp dq leak model")
    reconstructing_rsa_priv_key_iter(n, e, leaks)
    print()

    print("[+] Try iter version p q d leak model")
    leaks = rsa_random_leak_model((n, e, d, p, q, dp, dq, qinv), 0.43, 0.43, 0.43, 0.0, 0.0)
    reconstructing_rsa_priv_key_iter(n, e, leaks)
    print()
    
    print("[+] Try iter version p q leak model")
    leaks = rsa_random_leak_model((n, e, d, p, q, dp, dq, qinv), 0.52, 0.52, 0.0, 0.0, 0.0)
    (leak_d, leak_p, leak_q, leak_dp, leak_dq) = leaks
    reconstructing_rsa_priv_key_iter_pq(n, e, (leak_p, leak_q))
    print()
    # print("Try recursing version")
    # priv = reconstructing_rsa_priv_key(n, e, leaks)
    
test_reconstructing_rsa_priv_key()

[+] RSA Key information
[+] p = 112395907276255339298748538310065170806728520998241061618336577077854216045330352563597343088704681341603307214622129116570064459821633998460495609650142843838211120495184610814088008255519220553373760597911592123906905515594192797660686907668345297592827038519505252623518568887410118353170526576535257092261
[+] q = 146001821933461512332065834607391057243549221414058584317904557276262592813014587303467276757073247652091011849818591375498941682873680488982434530137291623755558925116431721003281201612570748447130158869240868225306023358958697894659848247852529248680569358811671550794211865590655039084685791700386329011487
[+] k = 7092
[+] kp = 38464
[+] kq = 3796

[+] Try iter version p q d dp dq leak model
[+] Recovered k = 7092
[+] Try with kp = 3796, kq = 38464
[+] Try with kp = 38464, kq = 3796
[+] RSA private key recovered !
[+] p = 112395907276255339298748538310065170806728520998241061618336577077854216045330352563597343088704681341603307214622129116

## Others

There is an open source project from the authors : https://hovav.net/ucsd/papers/hs09.html. This project is written in C and is fater than my implementation in sage. If you are dealing with the case $e$ with 32 bits or more, consider using the C project.
